In [1]:
fulldir="/Users/mahitha0x01/Documents/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/"
name="vid3/"

In [2]:
import shutil as sh
import cv2
import os
import numpy as np
from PIL import Image
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import random
from math import log
from tqdm import tqdm
import os.path

In [3]:
# file = open(fulldir+"pixels.txt","a+")

In [4]:
image = fulldir+name
ext = ".bmp"
key = "abcdefghijklm"
i=0

In [5]:
def getImageMatrix(imageName):
    im = Image.open(imageName, mode="r", formats=None) 
    pix = im.load()
    color = 1
    if type(pix[0,0]) == int:
        color=0
    image_size = im.size 
    image_matrix = []
    for width in range(int(image_size[0])):
        row = []
        for height in range(int(image_size[1])):
                row.append((pix[width,height]))
        image_matrix.append(row)
    return image_matrix,image_size[0], image_size[1],color

In [6]:
def LogisticEncryption(imageName, key):
    N = 256
    key_list = [ord(x) for x in key]
    G = [key_list[0:4] ,key_list[4:8], key_list[8:12]]
    g = []
    R = 1
    for i in range(1,4):
        s = 0
        for j in range(1,5):
            s += G[i-1][j-1] * (10**(-j))
        g.append(s)
        R = (R*s) % 1
    L = (R + key_list[12]/256) % 1
    S_x = round(((g[0]+g[1]+g[2])*(10**4) + L *(10**4)) % 256)
    V1 = sum(key_list)
    V2 = key_list[0]
    for i in range(1,13):
        V2 = V2 ^ key_list[i]
    V = V2/V1
    L_y = (V+key_list[12]/256) % 1
    S_y = round((V+V2+L_y*10**4) % 256)
    C1_0 = S_x
    C2_0 = S_y
    C = round((L*L_y*10**4) % 256)
    C_r = round((L*L_y*10**4) % 256)
    C_g = round((L*L_y*10**4) % 256)
    C_b = round((L*L_y*10**4) % 256)
    x = 4*(S_x)*(1-S_x)
    y = 4*(S_y)*(1-S_y)
    imageMatrix,dw, dl, color = getImageMatrix(imageName)
    LogisticEncryptionIm = []
    for i in range(dw):
        row = []
        for j in range(dl):
            max_iterations = 10
            iterations_x = 0
            iterations_y = 0
            while x <0.8 and x > 0.2 and iterations_x<=max_iterations:
                x = 4*x*(1-x)
                iterations_x+=1
            #print(iterations_x)
            while y <0.8 and y > 0.2 and iterations_y<=max_iterations:
                y = 4*y*(1-y)
                iterations_y+=1
            x_round = round((x*(10**4))%256)
            y_round = round((y*(10**4))%256)
            C1 = x_round ^ ((key_list[0]+x_round) % N) ^ ((C1_0 + key_list[1])%N)
            C2 = x_round ^ ((key_list[2]+y_round) % N) ^ ((C2_0 + key_list[3])%N) 
            if color:
                C_r =((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((key_list[6]+imageMatrix[i][j][0]) % N) ^ ((C_r + key_list[7]) % N)
                C_g =((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((key_list[6]+imageMatrix[i][j][1]) % N) ^ ((C_g + key_list[7]) % N)
                C_b =((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((key_list[6]+imageMatrix[i][j][2]) % N) ^ ((C_b + key_list[7]) % N)
                row.append((C_r,C_g,C_b))
                C = C_r
            else:
                C = ((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((key_list[6]+imageMatrix[i][j]) % N) ^ ((C + key_list[7]) % N)
                row.append(C)
            x = (x + C/256 + key_list[8]/256 + key_list[9]/256) % 1
            y = (x + C/256 + key_list[8]/256 + key_list[9]/256) % 1
            for ki in range(12):
                key_list[ki] = (key_list[ki] + key_list[12]) % 256
                key_list[12] = key_list[12] ^ key_list[ki]
        LogisticEncryptionIm.append(row)
    im = Image.new('L', (dw, dl))
    if color:
        im = Image.new('RGB', (dw, dl))
    else: 
        im = Image.new('L', (dw, dl)) # L is for Black and white pixels    
    pix = im.load()
    for x in range(dw):
        for y in range(dl):
            pix[x, y] = LogisticEncryptionIm[x][y]
    return im
    #im.save(imageName[0:-8]+"encryp"+ext, "BMP")

In [7]:
vals=[]

In [8]:
def EncryptionResult(img):
    cnt=0
    face = []
    modelFile = fulldir+"res10_300x300_ssd_iter_140000.caffemodel"
    configFile = fulldir+"deploy.prototxt.txt"
    net = cv2.dnn.readNetFromCaffe(configFile, modelFile)
    h, w = img.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(img, (800, 800)), 1.0,(800, 800), (104.0, 117.0, 123.0))
    net.setInput(blob)
    faces = net.forward()
    #to draw faces on image
    for i in range(faces.shape[2]):
        confidence = faces[0, 0, i, 2]
        if confidence > 0.5:
            box = faces[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x, y, x1, y1) = box.astype("int")
#           print(x,y,x1,y1)
            if y1<0: y1=0
            if x1<0: x1=0
            if x<0: x=0
            if y<0: y=0
            if y1>=h: y1=h-1
            if x1>=w: x1=w-1
            if x>=w: x=w-1
            if y>=h: y=h-1
            if x1-x>0 and y1-y>0:
                face.append((x, y, x1, y1))
                cv2.rectangle(img, (x, y), (x1-1, y1-1), (255, 0, 0), 1)
    count=0
    print(face)
    for (x, y, x1, y1) in face:
        cropped_image = img[y+1:y1-1, x+1:x1-1]
        filename=fulldir+"new/"+str(count)+ext
        cv2.imwrite(filename, cropped_image)
        imgt = LogisticEncryption(filename, key)
        LogisticResult=cv2.cvtColor(np.array(imgt),cv2.COLOR_RGB2BGR)
        img[y+1:y1-1, x+1:x1-1] = LogisticResult
        cv2.imwrite(fulldir+name+'/res.png', LogisticResult)
        count=count+1
    return img

In [9]:
# Read the video from specified path
cam = cv2.VideoCapture(fulldir+name+name[:-1]+".mp4")
fps = cam.get(cv2.CAP_PROP_FPS)
print(fps)

23.976023976023978


In [10]:
height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))

print(height, width)

720 1280


In [11]:
currentframe = 0
video_name = 'encryptedvideo.avi'
os.chdir(fulldir+name+"results")
video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'FFV1'), fps, (width, height))

while(True):
    ret,Img = cam.read()
    if ret:
        print("frame:",currentframe)
        frameout=EncryptionResult(Img)
        if currentframe<10:
            cv2.imwrite(fulldir+name+"frames/00"+str(currentframe)+".png", Img)
            cv2.imwrite(fulldir+name+"encrypted_frames/EncryptedResult00"+str(currentframe)+".bmp", frameout)
        elif currentframe<100:
            cv2.imwrite(fulldir+name+"frames/0"+str(currentframe)+".png", Img)
            cv2.imwrite(fulldir+name+"encrypted_frames/EncryptedResult0"+str(currentframe)+".bmp", frameout)
        else:
            cv2.imwrite(fulldir+name+"frames/"+str(currentframe)+".png", Img)
            cv2.imwrite(fulldir+name+"encrypted_frames/EncryptedResult"+str(currentframe)+".bmp", frameout)
        video.write(frameout)
        currentframe += 1
    else:
        break

frame: 0
[(142, 268, 235, 437), (754, 330, 842, 509)]
frame: 1
[(1025, 555, 1279, 719)]
frame: 2
[(118, 290, 219, 461), (584, 548, 860, 719)]
frame: 3
[(121, 304, 205, 484)]
frame: 4
[(112, 326, 203, 491), (60, 164, 234, 368)]
frame: 5
[(106, 329, 210, 494), (416, 115, 685, 344), (1043, 548, 1279, 719), (1102, 23, 1257, 266)]
frame: 6
[(102, 331, 218, 487), (1104, 23, 1259, 266)]
frame: 7
[(95, 317, 219, 468)]
frame: 8
[(79, 314, 204, 459), (695, 345, 798, 484)]
frame: 9
[(70, 296, 192, 446), (685, 349, 786, 486)]
frame: 10
[(52, 290, 179, 437), (1102, 23, 1260, 272)]
frame: 11
[(40, 285, 168, 431), (1110, 25, 1258, 269)]
frame: 12
[(21, 289, 153, 438)]
frame: 13
[(4, 301, 136, 451), (627, 318, 743, 468)]
frame: 14
[(2, 317, 120, 463)]
frame: 15
[(1, 330, 109, 480), (399, 187, 679, 428)]
frame: 16
[(629, 303, 725, 450), (403, 186, 685, 428), (1073, 557, 1279, 719)]
frame: 17
[(383, 118, 695, 339), (1089, 19, 1248, 263), (276, 244, 452, 507)]
frame: 18
[(1096, 19, 1247, 261), (385, 117,

frame: 118
[(150, 167, 245, 311)]
frame: 119
[(151, 172, 245, 320)]
frame: 120
[(155, 177, 247, 317), (1063, 549, 1279, 719)]
frame: 121
[(156, 172, 253, 311), (1067, 549, 1279, 719)]
frame: 122
[(153, 168, 249, 304)]
frame: 123
[(151, 160, 247, 300)]
frame: 124
[(150, 152, 247, 288)]
frame: 125
[(138, 143, 240, 278)]
frame: 126
[(131, 136, 231, 269)]
frame: 127
[(122, 133, 222, 270)]
frame: 128
[(114, 134, 213, 268)]
frame: 129
[(106, 142, 206, 273)]
frame: 130
[(98, 147, 194, 287)]
frame: 131
[(87, 156, 184, 296), (338, 298, 400, 393)]
frame: 132
[(78, 170, 176, 304)]
frame: 133
[(65, 177, 165, 312), (428, 194, 667, 450)]
frame: 134
[(40, 178, 152, 319), (415, 189, 669, 448), (308, 277, 387, 385), (3, 399, 104, 505), (1073, 543, 1279, 719)]
frame: 135
[(32, 172, 140, 314), (4, 384, 105, 523), (295, 267, 387, 396)]
frame: 136
[(17, 161, 127, 307), (3, 370, 101, 519)]
frame: 137
[(5, 153, 124, 301), (3, 352, 95, 501), (403, 188, 666, 442), (285, 265, 365, 371)]
frame: 138
[(4, 149, 110

frame: 237
[(251, 199, 350, 343), (486, 223, 580, 356), (393, 97, 655, 345), (1081, 542, 1279, 719), (1061, 306, 1279, 536)]
frame: 238
[(233, 187, 336, 330), (471, 229, 573, 358), (398, 93, 653, 336), (1079, 543, 1279, 719)]
frame: 239
[(226, 174, 330, 317), (468, 235, 567, 363), (402, 93, 655, 335)]
frame: 240
[(221, 158, 324, 300), (454, 236, 557, 362)]
frame: 241
[(214, 148, 317, 291), (451, 238, 551, 364)]
frame: 242
[(209, 138, 312, 284), (446, 231, 542, 360)]
frame: 243
[(204, 141, 305, 286), (440, 224, 536, 352)]
frame: 244
[(201, 143, 305, 292), (443, 214, 533, 340)]
frame: 245
[(198, 154, 301, 303), (440, 210, 532, 339), (890, 458, 1051, 716)]
frame: 246
[(190, 167, 294, 318), (433, 212, 524, 345)]
frame: 247
[(188, 181, 293, 329), (434, 216, 524, 349)]
frame: 248
[(190, 190, 291, 341), (427, 224, 521, 357), (1056, 544, 1279, 719)]
frame: 249
[(191, 191, 293, 342), (428, 235, 522, 368)]
frame: 250
[(195, 188, 296, 336), (424, 245, 521, 374)]
frame: 251
[(196, 177, 298, 330), 

frame: 361
[(38, 649, 102, 712), (186, 229, 282, 345), (408, 113, 675, 351)]
frame: 362
[(185, 240, 282, 356), (38, 650, 103, 713), (404, 115, 676, 354)]
frame: 363
[(184, 252, 283, 366), (38, 643, 103, 709)]
frame: 364
[(182, 254, 282, 367), (29, 634, 100, 699), (406, 186, 656, 432)]
frame: 365
[(181, 250, 285, 364), (21, 620, 95, 689), (407, 185, 660, 431)]
frame: 366
[(180, 243, 281, 356)]
frame: 367
[(175, 235, 276, 349), (5, 596, 75, 667)]
frame: 368
[(167, 223, 273, 336), (3, 589, 74, 660)]
frame: 369
[(161, 208, 264, 323), (402, 112, 677, 346), (0, 589, 67, 660)]
frame: 370
[(151, 198, 254, 319), (1063, 553, 1279, 719), (379, 113, 687, 337)]
frame: 371
[(147, 193, 250, 313), (385, 110, 690, 336)]
frame: 372
[(144, 197, 244, 315), (1060, 550, 1279, 719)]
frame: 373
[(140, 203, 244, 326)]
frame: 374
[(137, 218, 239, 341)]
frame: 375
[(128, 231, 229, 354)]
frame: 376
[(122, 244, 221, 364)]
frame: 377
[(114, 250, 213, 370)]
frame: 378
[(101, 244, 200, 367)]
frame: 379
[(93, 238, 188

frame: 507
[(173, 382, 270, 511), (397, 191, 652, 453), (1085, 551, 1279, 719)]
frame: 508
[(178, 386, 277, 510), (886, 263, 993, 386)]
frame: 509
[(183, 381, 278, 507)]
frame: 510
[(183, 376, 278, 501), (1066, 549, 1279, 719)]
frame: 511
[(186, 370, 279, 496), (1072, 549, 1279, 719)]
frame: 512
[(183, 366, 279, 490), (1073, 547, 1279, 719)]
frame: 513
[(178, 364, 274, 490), (426, 191, 666, 448)]
frame: 514
[(176, 365, 271, 490), (423, 191, 673, 443)]
frame: 515
[(172, 365, 267, 494), (850, 260, 950, 386), (413, 190, 674, 439)]
frame: 516
[(167, 371, 263, 500), (412, 189, 678, 434), (1064, 556, 1279, 719)]
frame: 517
[(163, 377, 260, 507), (417, 189, 684, 432), (1057, 554, 1279, 719), (829, 273, 939, 388)]
frame: 518
[(156, 383, 254, 515), (422, 188, 684, 431), (1058, 553, 1279, 719)]
frame: 519
[(149, 387, 252, 520), (406, 112, 676, 350), (1062, 554, 1279, 719)]
frame: 520
[(146, 391, 247, 518), (1064, 552, 1279, 719), (400, 109, 673, 348)]
frame: 521
[(148, 393, 245, 517), (1069, 549

frame: 602
[(471, 252, 575, 391), (419, 105, 685, 352), (246, 216, 343, 341), (1002, 196, 1092, 344)]
frame: 603
[(459, 263, 563, 403), (246, 209, 339, 335), (424, 102, 693, 351), (1076, 544, 1279, 719)]
frame: 604
[(445, 275, 553, 414), (244, 207, 342, 334), (423, 102, 690, 348)]
frame: 605
[(432, 275, 535, 415), (244, 212, 344, 339), (424, 105, 686, 345), (1093, 286, 1185, 410)]
frame: 606
[(421, 274, 526, 415), (245, 220, 342, 349), (1092, 288, 1184, 415)]
frame: 607
[(411, 270, 518, 408), (247, 228, 343, 358), (1090, 296, 1184, 428)]
frame: 608
[(404, 260, 511, 399), (245, 239, 343, 366), (1088, 311, 1182, 442)]
frame: 609
[(399, 249, 506, 389), (244, 245, 346, 369), (1088, 315, 1180, 450)]
frame: 610
[(394, 244, 502, 384), (246, 242, 346, 367), (1086, 315, 1175, 452), (950, 201, 1047, 369)]
frame: 611
[(393, 245, 501, 384), (247, 235, 349, 361), (952, 196, 1046, 358), (1086, 312, 1176, 451)]
frame: 612
[(389, 248, 499, 390), (247, 229, 346, 354), (418, 188, 672, 430), (948, 194, 1

In [12]:
cv2.destroyAllWindows()
video.release()